# Общее

In [ ]:
!pip install scikit-surprise

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy
from google.colab import drive
from surprise import KNNBaseline
from surprise import Dataset
from surprise import Reader

In [ ]:
drive.mount('/content/drive/')
# products = pd.read_csv('/content/drive/My Drive/Курсовые/5_products.csv', nrows=100000)
transactions = pd.read_csv('/content/drive/My Drive/Курсовые/5_transactions.csv', nrows=3000000)
products = pd.read_csv('/content/drive/My Drive/Курсовые/5_products.csv')
# transactions = pd.read_csv('/content/drive/My Drive/Курсовые/5_transactions.csv')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


products.csv

product_id - уникальный идентификатор товара

product_name - название товара

aisle_id - уникальный идентификатор подкатегории

department_id - уникальный идентификатор категории

aisle - название подкатегории

department - название категории

transactions.csv

order_id - уникальный идентификатор транзакции

user_id - уникальный идентификатор покупателя

order_number - номер транзакции в истории покупок данного пользователя

order_dow - день недели транзакции

order_hour_of_day - час совершения транзакции

days_since_prior_order - количество дней с совершения предыдущей
транзакции данным пользователем

product_id - уникальный идентификатор товара

add_to_cart_order - номер под которым данный товар был добавлен в корзину

reordered - был ли товар "перезаказан"

In [ ]:
data = transactions.merge(products, on='product_id')

In [ ]:
from surprise.model_selection import train_test_split

In [ ]:
rating = dict(data['product_id'].value_counts())

In [ ]:
rat_list = []
for i in data['product_id']:
  rat_list.append(rating[i])

In [ ]:
data['rating'] = rat_list

In [ ]:
data = data[['user_id', 'product_id', 'rating']]

In [ ]:
data['userprod'] = data['user_id'].astype(str) + data['product_id'].astype(str)

In [ ]:
data.head()

,user_id,product_id,rating,userprod
0,1,196,3259,1196
1,1,196,3259,1196
2,1,196,3259,1196
3,1,196,3259,1196
4,1,196,3259,1196


In [ ]:
len(data)

3000000

In [ ]:
data.drop_duplicates(subset=['userprod'], inplace=True)

In [ ]:
data.head()

,user_id,product_id,rating,userprod
0,1,196,3259,1196
10,15,196,3259,15196
14,21,196,3259,21196
15,31,196,3259,31196
17,43,196,3259,43196


In [ ]:
len(data)

1071023

In [ ]:
data = data[['user_id', 'product_id', 'rating']]

In [ ]:
data.head()

,user_id,product_id,rating
0,1,196,3259
10,15,196,3259
14,21,196,3259
15,31,196,3259
17,43,196,3259


In [ ]:
len(data)

1071023

In [ ]:
data.groupby(['user_id', 'product_id']).value_counts().sum()

1071023

In [ ]:
data.groupby(['user_id', 'product_id']).nunique().sum()

rating    1071023
dtype: int64

## Ридеры и обучение

In [ ]:
reader = Reader(rating_scale=(data.rating.min(), data.rating.max()))

In [ ]:
data_set = Dataset.load_from_df(data[['user_id', 'product_id', 'rating']], reader)

In [ ]:
trainset, testset = train_test_split(data_set, test_size=0.2)  # разделение на обучающую и тестовую выборки

algo = KNNBaseline()
algo.fit(trainset)
predictions = algo.test(testset)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.


In [ ]:
predictions[0]

Prediction(uid=17050, iid=8075, r_ui=231.0, est=181.41038344974214, details={'actual_k': 40, 'was_impossible': False})

In [ ]:
df_ratings = pd.DataFrame(columns=['userID', 'itemID', 'rating'])

In [ ]:
for i in predictions:
  df_ratings = df_ratings.append({'userID':i.uid, 'itemID':i.iid, 'rating':i.est}, ignore_index=True)

Выходные данные были обрезаны до нескольких последних строк (5000).
<ipython-input-27-140bb75e2b8b>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_ratings = df_ratings.append({'userID':i.uid, 'itemID':i.iid, 'rating':i.est}, ignore_index=True)
<ipython-input-27-140bb75e2b8b>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_ratings = df_ratings.append({'userID':i.uid, 'itemID':i.iid, 'rating':i.est}, ignore_index=True)
<ipython-input-27-140bb75e2b8b>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_ratings = df_ratings.append({'userID':i.uid, 'itemID':i.iid, 'rating':i.est}, ignore_index=True)
<ipython-input-27-140bb75e2b8b>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in

In [ ]:
df_ratings.head()

,userID,itemID,rating
0,17050.0,8075.0,181.410383
1,22842.0,48679.0,4623.946348
2,21028.0,19068.0,552.405396
3,1041.0,663.0,14.825436
4,14306.0,30305.0,919.952741


In [ ]:
def recomended(data, user):
  return [i for i in data[data['userID'] == user].sort_values(by='rating')['itemID'][:10]]

In [ ]:
recommended_dict = {}
for i in df_ratings.userID.unique():
  recommended_dict[i] = recomended(df_ratings, i)

In [ ]:
recommended_dict[154]

[44204.0,
 31184.0,
 39764.0,
 12381.0,
 47392.0,
 16494.0,
 7189.0,
 30620.0,
 35093.0,
 13391.0]

In [ ]:
df = pd.DataFrame({'userID':recommended_dict.keys(), 'itemID':recommended_dict.values()})
df.head()

,userID,itemID
0,17050.0,"[37168.0, 566.0, 4473.0, 49100.0, 32869.0, 846..."
1,22842.0,"[17590.0, 19104.0, 28683.0, 36554.0, 43338.0, ..."
2,21028.0,"[22857.0, 29092.0, 41750.0, 47804.0, 19025.0, ..."
3,1041.0,"[42075.0, 663.0, 6218.0, 20015.0, 26668.0, 290..."
4,14306.0,"[16715.0, 13351.0, 26842.0, 44580.0, 4796.0, 4..."


In [ ]:
df.to_csv(r' E:\my_data.csv', index=False)